In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.layers import GlobalAveragePooling1D
import yfinance as yf

In [33]:
data = yf.download('BTC-USD', start="2014-09-17", end="2024-05-19")

[*********************100%%**********************]  1 of 1 completed


In [34]:
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data[['High', 'Low', 'Close']].values)

In [35]:
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step):
        dataX.append(dataset[i:(i+time_step), :])
        dataY.append(dataset[i + time_step, :])
    return np.array(dataX), np.array(dataY)

In [37]:
time_step = 30
dataX, datay = create_dataset(data_scaled, time_step)

In [44]:
x_train, x_test, y_train, y_test = train_test_split(dataX, datay, test_size = 0.1, random_state=42)

In [45]:
x_train =x_train.reshape(x_train.shape[0],x_train.shape[1] , 3) #sample, time step, features
x_test = x_test.reshape(x_test.shape[0],x_test.shape[1] , 3)
print("X_train: ", x_train.shape)
print("X_test: ", x_test.shape)

X_train:  (3150, 30, 3)
X_test:  (351, 30, 3)


In [47]:
def create_lstm_model():
    model = Sequential()
    model.add(LSTM(128, activation='gelu', dropout=0.0, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
    model.add(LSTM(64, activation='gelu', dropout=0.2))
    model.add(Dense(64))
    model.add(Dense(32))
    model.add(Dense(3))
    return model

In [48]:
model = create_lstm_model()
model.summary()
model.compile(optimizer='adam',
    loss='mean_squared_error')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 128)           67584     
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 3)                 99        
                                                                 
Total params: 123,331
Trainable params: 123,331
Non-trainable params: 0
_________________________________________________________________


In [50]:
checkpoint_filepath = 'model_{epoch}.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_filepath,
    save_weights_only = True,
    monitor = 'val_loss',
    mode='min',
    save_best_only=True
)

model_earlystopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience = 10,
    verbose = 1, #shows message whnen callback takes an action
    mode = 'min',
    restore_best_weights = True
)

history = model.fit(x_train,
    y_train,
    validation_data=(x_test,y_test),
    batch_size=32,
    epochs=100,
    verbose=1,
    callbacks = [model_checkpoint_callback]
)

Epoch 1/100
99/99 [==============================] - 18s 115ms/step - loss: 0.0147 - val_loss: 9.2530e-04
Epoch 2/100
99/99 [==============================] - 9s 88ms/step - loss: 8.5665e-04 - val_loss: 0.0014
Epoch 3/100
99/99 [==============================] - 12s 126ms/step - loss: 8.4867e-04 - val_loss: 9.3700e-04
Epoch 4/100
99/99 [==============================] - 12s 126ms/step - loss: 7.6500e-04 - val_loss: 7.6607e-04
Epoch 5/100
99/99 [==============================] - 11s 114ms/step - loss: 7.6966e-04 - val_loss: 0.0012
Epoch 6/100
99/99 [==============================] - 9s 92ms/step - loss: 7.1468e-04 - val_loss: 8.7699e-04
Epoch 7/100
99/99 [==============================] - 11s 106ms/step - loss: 6.0445e-04 - val_loss: 5.7705e-04
Epoch 8/100
99/99 [==============================] - 12s 120ms/step - loss: 5.7711e-04 - val_loss: 5.1415e-04
Epoch 9/100
99/99 [==============================] - 13s 128ms/step - loss: 6.6801e-04 - val_loss: 4.7296e-04
Epoch 10/100
99/99 [======

In [51]:
model.save("model.keras")

In [52]:
training_prediction = model.predict(x_train)
train_score = mean_squared_error(y_train, training_prediction)
train_score_unscaled = mean_squared_error(scaler.inverse_transform(y_train), scaler.inverse_transform(training_prediction))
print(train_score)
print(train_score_unscaled)

99/99 [==============================] - 3s 26ms/step
0.00012861760575866524
676402.9950016916


In [53]:
test_prediction = model.predict(x_test)
test_score = mean_squared_error(y_test, test_prediction)
test_score_unscaled = mean_squared_error(scaler.inverse_transform(y_test), scaler.inverse_transform(test_prediction))
print(test_score)
print(test_score_unscaled)

11/11 [==============================] - 0s 24ms/step
0.00011503550238739137
605627.5606942744


In [55]:
#r-2 score
train_r2 = r2_score(scaler.inverse_transform(training_prediction), (scaler.inverse_transform(y_train)))
print(train_r2)
test_r2 = r2_score(scaler.inverse_transform(test_prediction), (scaler.inverse_transform(y_test)))
print(test_r2)

0.9978796336344445
0.998076771691968
